In [2]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [3]:
# Sep_deacts=pd.read_csv('SMHM_Deacts_Sep2022.csv')

In [4]:
Nov_deacts=pd.read_csv('SMHH_CHURN_DEC2022.csv')

In [5]:
Nov_deacts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4229 entries, 0 to 4228
Data columns (total 43 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ACTIVITY_DATE                4229 non-null   object 
 1   CUST_ID                      4229 non-null   int64  
 2   BILLING_ACCOUNT_NUM          4229 non-null   int64  
 3   BUS_BILLING_ACCOUNT_NUM      4229 non-null   int64  
 4   BUS_PROD_INSTNC_ID           4229 non-null   int64  
 5   ACTIVITY_REASON_TXT          4229 non-null   object 
 6   ACTIVITY_REASON_TYPE_CD      4229 non-null   object 
 7   ACTIVITY_REASON_SUB_TYPE_CD  4229 non-null   object 
 8   INITIAL_ACTIVATION_TS        4229 non-null   object 
 9   CONTRACT_START_DT            3552 non-null   object 
 10  CONTRACT_END_DT              3552 non-null   object 
 11  TERM_LENGTH_NUM              4229 non-null   int64  
 12  PREFERRED_LANGUAGE_TXT       4229 non-null   object 
 13  BUSINESS_LINE     

In [6]:
Nov_deacts['Deact_Flag']=1

In [7]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_DEACTS_DEC2022'

bq_table_instance= bq_client.load_table_from_dataframe(Nov_deacts, Table_BQ,job_config=config)